# Lexing

In [1]:
from enum import Enum, auto

In [2]:
class Token:
    class Type(Enum):
        INTEGER = auto()
        PLUS = auto()
        MINUS = auto()
        LPAREN = auto()
        RPAREN = auto()

    def __init__(self, type, text):
        self.type = type
        self.text = text

    def __str__(self):
        return f'`{self.text}`'


In [3]:
def lex(input):
    result = []

    i = 0
    while i < len(input):
        if input[i] == '+':
            result.append(Token(Token.Type.PLUS, '+'))
        elif input[i] == '-':
            result.append(Token(Token.Type.MINUS, '-'))
        elif input[i] == '(':
            result.append(Token(Token.Type.LPAREN, '('))
        elif input[i] == ')':
            result.append(Token(Token.Type.RPAREN, ')'))
        else:  # must be a number
            digits = [input[i]]
            for j in range(i + 1, len(input)):
                if input[j].isdigit():
                    digits.append(input[j])
                    i += 1
                else:
                    result.append(Token(Token.Type.INTEGER,
                                        ''.join(digits)))
                    break
        i += 1

    return result

In [4]:
def calc(input):
  tokens = lex(input)
  print(' '.join(map(str, tokens)))


In [5]:
calc('(13+4)-(12+1)')

`(` `13` `+` `4` `)` `-` `(` `12` `+` `1` `)`


# Parsing

In [6]:
class Integer:
    def __init__(self, value):
        self.value = value

In [7]:
class BinaryOperation:
    class Type(Enum):
        ADDITION = auto()
        SUBTRACTION = auto()

    def __init__(self):
        self.type = None
        self.left = None
        self.right = None

    @property
    def value(self):
        if self.type == self.Type.ADDITION:
            return self.left.value + self.right.value
        elif self.type == self.Type.SUBTRACTION:
            return self.left.value - self.right.value

In [8]:
def parse(tokens):
    result = BinaryOperation()
    # lhs = left hand side
    have_lhs = False
    i = 0
    while i < len(tokens):
        token = tokens[i]

        if token.type == Token.Type.INTEGER:
            integer = Integer(int(token.text))
            if not have_lhs:
                result.left = integer
                have_lhs = True
            else:
                result.right = integer
        elif token.type == Token.Type.PLUS:
            result.type = BinaryOperation.Type.ADDITION
        elif token.type == Token.Type.MINUS:
            result.type = BinaryOperation.Type.SUBTRACTION
        elif token.type == Token.Type.LPAREN:  # note: no if for RPAREN
            j = i
            while j < len(tokens):
                if tokens[j].type == Token.Type.RPAREN:
                    break
                j += 1
            # preprocess subexpression
            subexpression = tokens[i + 1:j]
            element = parse(subexpression)
            if not have_lhs:
                result.left = element
                have_lhs = True
            else:
                result.right = element
            i = j  # advance
        i += 1
    return result

In [9]:
def calc(input):
  tokens = lex(input)
  # print(' '.join(map(str, tokens)))

  # parse into binary tree
  parsed = parse(tokens)
  print(f'{input} = {parsed.value}')


In [10]:
calc('(13+4)-(12+1)')

(13+4)-(12+1) = 4


In [11]:
calc('1+(3-4)')

1+(3-4) = 0


In [12]:
# this won't work
calc('1+2+(3-4)')

1+2+(3-4) = 0
